<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9D%B4%EC%86%8C%ED%98%84_%EC%84%9C%EB%B9%84%EC%8A%A42_%EB%AA%A8%EB%8D%B8%EB%A7%81%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
import re

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/Final_Project/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# CSV 파일 불러오기

In [ ]:
df = pd.read_csv(f"{DATA_PATH}train_0920.csv")
df

,고객리뷰,사장댓글,sentiment,food,store,branch
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,김치,NaN,NaN
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,,NaN,NaN
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,,NaN,NaN
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,잡채,NaN,NaN
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,,NaN,NaN
...,...,...,...,...,...,...
18641,맛있어요 배달도 빠르고 국물도 안새고 안전배달 잘먹었습니다,감사드립니다 리뷰님. 찬바람 건강 유의하세요,1,,,
18642,주말되면 생각나서 찾게되는 너무맛있어요 잘먹었습니다~,저희 인생 아귀찜은 신선한 재료와 넉넉한 양으로 고객님께 맛있는 식사를 대접해 드리...,1,아귀,인생 아귀찜,
18643,맛있게 잘먹었네요. 배부르네요,맛있게 드셔주셔서 감사합니다,1,,,
18644,맛있게 잘 먹었습니다,맛있게 잘 드셨다니 감사합니다~ 다음에 드실 때도 변함없이 맛있게 드실 수 있도록 ...,1,닭,,


In [ ]:
df['store'] = df['store'].fillna("")
df['branch'] = df['branch'].fillna("")

In [ ]:
df.head()

,고객리뷰,사장댓글,sentiment,food,store,branch
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,김치,,
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,,,
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,,,
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,잡채,,
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,,,


### 긍정/부정 비율

In [ ]:
for i in range(0, 2):
    num_rating = df[df['sentiment'] == i].shape[0]
    print(f"긍/부정 {i}점 개수: {num_rating}, {num_rating / df.shape[0] * 100}%")

긍/부정 0점 개수: 9323, 50.0%
긍/부정 1점 개수: 9323, 50.0%


# 모델 불러오기

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00


In [ ]:
model_name = "skt/kogpt2-base-v2"
# model_name = "EasthShin/Youth_Chatbot_Kogpt2-base"

In [ ]:
# AutoModelForCausalLM 인과 관련 사용
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

# 데이터셋 만들기

In [ ]:
df.head()

,고객리뷰,사장댓글,sentiment,food,store,branch
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,김치,,
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,,,
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,,,
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,잡채,,
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,,,


In [ ]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        sentiment = item["sentiment"]
        store = item["store"]
        branch = item["branch"]
        food = item["food"]
        review = item["고객리뷰"]
        # review = item["keyword"]
        answer = item["사장댓글"]

        return {
            "sentiment": sentiment,
            "store": store,
            "branch": branch,
            "food": food,
            "review": review,
            "answer": answer
        }

In [ ]:
def collate_fn(batch):
    sentiment = torch.tensor([item["sentiment"] for item in batch])
    texts = [
        f"<w>{item['store']}</s><b>{item['branch']}</s><f>{item['food']}</s><q>{item['review']}</s><a>{item['answer']}</s>"
        for item in batch
    ]
    x = tokenizer(texts, return_tensors="pt", padding=True)

    return {"x": x, "sentiment": sentiment}

In [ ]:
dt = ChatDataset(df)
dt[0]

{'sentiment': 0,
 'store': '',
 'branch': '',
 'food': '김치',
 'review': '김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고. ',
 'answer': '안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 위해 노력하겠습니다. 감사합니다.'}

In [ ]:
dt = ChatDataset(df)
dl = torch.utils.data.DataLoader(dt,batch_size=2,collate_fn=collate_fn)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[ 9724,   461,   405,     1,  9724,   440,   405,     1,  9724,   444,
            405,  6963,  8420,     1,  9724,   455,   405,  6963,  8420,  8326,
          19010, 15126,  8041,  8006, 25856, 10968, 33974,  6920, 31131, 13875,
          40974,  8718,  8006, 25856,  9065,  9313, 15126, 19570,   389,   739,
              1,  9724,   439,   405,  7967,  7114,  8702,  7801, 25856, 33974,
           8326, 24655,  9065,  9399,  9167,  9137, 12615,  9535,  7415, 11482,
           7832, 37194, 18323, 14912, 32272,  9207, 10805, 26616, 16691, 15940,
          37194,     1],
         [ 9724,   461,   405,     1,  9724,   440,   405,     1,  9724,   444,
            405,   739,     1,  9724,   455,   405, 13692, 12371, 27080,  9208,
           8539,  8267,  7724,  8718,  8006, 25856, 10997,  7235, 15126,  8041,
           8006, 25856,   739,     1,  9724,   439,   405,  7609,  8539,  8267,
           7724,  8702,  7816, 16179, 29205, 24721,  6975, 16691, 37324,  91

In [ ]:
model(**batch["x"]).logits.shape

torch.Size([2, 72, 51200])

# 학습루프

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):

        x = batch["x"].to(device)
        sentiment = batch["sentiment"].to(device)

        pred = model(**x).logits  # batch, seq, 단어별 실수값
        n_class = pred.shape[-1]  # 클래스 개수
        sentiment_pred = pred[:, -1, :]

        pred = pred[:, :-1, :]  # eos 토큰 제외
        pred = pred.reshape(-1, n_class)  # 2차원 형태로 변환

        tgt = x["input_ids"][:, 1:]
        tgt = tgt.flatten()  # 1차원 벡터 형태로 바뀜

        mask = tgt != tokenizer.pad_token_id  # 패딩토큰 제외 -> 손실을 계산할 때 제외
        tgt = tgt[mask]
        pred = pred[mask]

        loss = loss_fn(pred, tgt)

        sentiment_loss = loss_fn(sentiment_pred, sentiment)

        optimizer.zero_grad()
        total_loss = loss + sentiment_loss
        total_loss.backward()
        optimizer.step()

        epoch_loss += total_loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [ ]:
reset_seeds(SEED)
batch_size = 2
epochs = 5

# model_name = "EasthShin/Youth_Chatbot_Kogpt2-base"
model_name = "skt/kogpt2-base-v2"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

## 동결 코드 ###
# for name,param in model.named_parameters():
#     if name.startswith("transformer.h.4"):
#         break
#     param.requires_grad = False

optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
loss_fn = torch.nn.CrossEntropyLoss()

train_dt = ChatDataset(df)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for i in range(epochs):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
    print(f"{i+1}번째 epoch: {train_loss:.4f}")


  0%|          | 0/9323 [00:00<?, ?it/s]

1번째 epoch: 1.4962


  0%|          | 0/9323 [00:00<?, ?it/s]

2번째 epoch: 1.1713


  0%|          | 0/9323 [00:00<?, ?it/s]

3번째 epoch: 1.0393


  0%|          | 0/9323 [00:00<?, ?it/s]

4번째 epoch: 0.9410


  0%|          | 0/9323 [00:00<?, ?it/s]

5번째 epoch: 0.8486


- 0905_csv 5000개 epochs = 10, 0.6046
- 0906_csv sentiment:0(부정) batch = 2, epochs = 20, 0.4039 (18->0.3136)
- 0906_csv sentiment:1(긍정) batch = 2, epochs = 10, 0.4291
- 0907_df_neg_augmented.csv batch = 2, epochs = 15, 0.2129(2724개)
- 0908_ng_df.csv batch = 2, epochs = 18, 0.2032(1483개)
- 0908_ps_df.csv batch = 2, epochs = 15, 0.2152(3000개)
- gpt_neg_reviews_4400(부정) + train+sentiment_0913(긍정) batch = 2, epochs = 10, 0.4319(8781개)
- 위와 동일한 dataset, transformer.h.8까지 동결 epochs = 10, 0.8454
- 위와 동일한 dataset, transformer.h.10까지 동결 epochs = 10, 1.1938
- 위와 동일한 dataset, transformer.h.3까지 동결 epochs = 10, 0.5480
- gpt_neg_reviews_9300(부정) + train+sentiment_0913(긍정) batch = 2, epochs = 10, 0.5088(skt)
- gpt_neg_reviews_9300(부정) + train+sentiment_0913(긍정) batch = 2, epochs = 10, 0.4975(youth)
- gpt_neg_reviews_9300(부정) + train+sentiment_0913(긍정) batch = 2, epochs = 5, 0.8486(skt)

# 학습모델 저장

In [ ]:
model.save_pretrained(f"{DATA_PATH}0920_skt_gpt")

# 저장한 모델 로드

In [ ]:
loaded_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}0919_youth_gpt").to(device)

## 맞춤법/띄어쓰기 검사

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-hfw5h43i
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-hfw5h43i
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4813 sha256=f63a3d614e1058592122852e0e383f803b0e9a01d70bd2ea1187c257afec099c
  Stored in directory: /tmp/pip-ephem-wheel-cache-4a3me4i_/wheels/2e/43/cc/753c9e1d91affb9ea40e186cea5654fb9231deb454da6724e5
Successfully built py-hanspell


In [ ]:
from hanspell import spell_checker

def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

## Paraphrase

In [ ]:
# 옵션
from transformers import pipeline
pipe = pipeline("text2text-generation", model="lcw99/t5-base-korean-paraphrase")

def paraphrase_text(text):
    generated_text = pipe(text, max_length=512, early_stopping=True,
                          num_beams=4,
                          num_return_sequences=2,
                          no_repeat_ngram_size=4,
                          top_k=50,
                          top_p=0.95)
    generated_text = generated_text[0]['generated_text']
    return generated_text

## 유사도

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4f0ae2e9cf1995e1915b2a320e4c9817ee468da64f2d2ed8db28c5238f870df5
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score
bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def evaluate_similarity(input_text, generated_text, alpha=0.9): # cosine 가중치 조절
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))
    # jaccard_sim = jaccard_score(input_tokens, generated_tokens)

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

In [ ]:
evaluate_similarity("맛은 있는데 양이 적어요",'맛있게 드셔주셔서 감사합니다. 다음에 양 많이 드릴게요')

0.3582301497459412

## 테스트루프

### user> 직접 입력 -> 유사도 + (맞춤법검사)

In [ ]:
def chatbot_with_evaluation(model, tokenizer, max_len, device, num_samples=3):
    model.eval()
    while True:
        text = input("user > ").strip()
        if text == "quit":
            break

        text = "<q>" + text + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        best_generated_text = None
        best_similarity_score = -1.0
        generated_texts = []  # 답변 후보군을 담을 리스트
        for i in range(num_samples):
            result_ids = model.generate(x,
                                        max_length=max_len,
                                        repetition_penalty=2.0,
                                        use_cache=True,
                                        do_sample=True,
                                        temperature=0.8,
                                        top_k=10, #50
                                        num_return_sequences=1
                                        )
            generated_text = tokenizer.decode(result_ids[0])
            # correct_spell(generated_text) # 맞춤법 검사
            generated_text = generated_text[q_len:-4]
            # generated_text = paraphrase_text(generated_text)
            generated_text = re.sub(r'[^가-힣.]', ' ', generated_text)

            similarity_score = evaluate_similarity(text, generated_text)
            generated_texts.append((similarity_score,generated_text))
            print(generated_texts[i])

            if similarity_score > best_similarity_score:
                best_similarity_score = similarity_score
                best_generated_text = generated_text

        print()
        print("Best_reply >", best_generated_text)
        print("Best_similarity_score:", best_similarity_score)
        print("--------------------------------------------------------------------------------------------")


In [ ]:
chatbot_with_evaluation(model,tokenizer,128,device)

user > 맛있어요
(0.3516654431819916, '감사합니다 고객님  다음에도 꼭 만족을 드릴게요.       많이 사랑해 주세요  또 이용해 주실 거죠  ')
(0.40611133575439456, '감사합니다 고객님  다음에도 꼭 봉 내를 찾아주세요. 맛있게 해드릴게요       요 새해 복 많이 받으시고 항상 건강하시고 행복하시는 날 되시길 바라겠습니다. 많은 사랑 부탁드려요  감사드리고  또 만나 주실 거죠   행복한 저녁시간 보내셔서 리뷰 남겨봅니다. 저희도 올 한 해도 좋은 일 가득이셨으면 좋았는데. 따뜻한 하루가 되었기를 바랍니까다  파이팅 하십시오 ')
(0.4357371926307678, '이용해 주시고 맛있게 드셔주시다니 감사드립니다 즐거운 저녁시간 보내세요.      저희도 행복한 마음으로 퇴근합니다.. 고객님께서 남겨주도신 리뷰  감기 조심하시고 편안한 하루하루 보낼게용. 무더위에 건강 유의하십니까 다음에 또 주문 부탁드려요  이용해 주세영부탁 드릴 따름이 되겠습니다. 늘 최선을 다하겠습니다  애용 바랍니가 좋네고  행복의 밤 보내시기 좋은 거에 용 즐겁고 건강한 한 주도 파이팅 하')

Best_reply > 이용해 주시고 맛있게 드셔주시다니 감사드립니다 즐거운 저녁시간 보내세요.      저희도 행복한 마음으로 퇴근합니다.. 고객님께서 남겨주도신 리뷰  감기 조심하시고 편안한 하루하루 보낼게용. 무더위에 건강 유의하십니까 다음에 또 주문 부탁드려요  이용해 주세영부탁 드릴 따름이 되겠습니다. 늘 최선을 다하겠습니다  애용 바랍니가 좋네고  행복의 밤 보내시기 좋은 거에 용 즐겁고 건강한 한 주도 파이팅 하
Best_similarity_score: 0.4357371926307678
--------------------------------------------------------------------------------------------
user > 맛없어요
(0.12301720976829529, '감사합니다 고객님  